In [3]:
import pandas as pd

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)


/Users/rajshekhar/Desktop/ci/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
df = pd.read_csv('def_api.csv')
df

,classes,meainings
0,GetCurrentObject,GetCurrentObject function (wingdi.h) The GetCu...
1,ExitThread,Application.ExitThread Method Definition Names...
2,IsMenu,IsMenu function (winuser.h) Determines whether...
3,WSALookupServiceBeginA,WSALookupServiceBeginA function (winsock2.h) T...
4,DebugActiveProcess,DebugActiveProcess function (debugapi.h) Enabl...
...,...,...
661,SetFilePointer,Positioning a File Pointer When an application...
662,GetVolumeInformationA,GetVolumeInformationA function (fileapi.h) Ret...
663,GetTimeFormatA,GetTimeFormatA function (datetimeapi.h) Format...
664,102,MSSQLSERVER_102 Applies to: SQL Server Azure S...


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class WordMeaningDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data[idx]["class"]
        meaning = self.data[idx]["meaning"]

        # Tokenize word and meaning together
        inputs = self.tokenizer(word, meaning, 
                                padding="max_length", 
                                truncation=True, 
                                max_length=self.max_length, 
                                return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
        }
    # def __getitem__(self, idx):
    #     try:
    #         item = self.data[idx]  # Attempt to access the item at index `idx`
            
    #         # Debug: Print type of item to verify expected format
    #         print(f"Processing index: {idx}, Type: {type(item)}")

    #         # Ensure the item is a dictionary before accessing keys
    #         if not isinstance(item, dict):
    #             raise TypeError(f"Expected dictionary but got {type(item)} at index {idx}")

    #         word = item.get("class", "Unknown")  # Use .get() to avoid KeyErrors
    #         meaning = item.get("meaning", "No meaning found")

    #         return word, meaning

    #     except Exception as e:
    #         print(f"Error at index {idx}: {e}")  # Print error message
    #         print(f"Batch content at index {idx}: {self.data[idx]}")  # Show the problematic batch
    #         return None  # You can return None or handle it differently
    # def __getitem__(self, idx):
    #     try:
    #         item = self.data[idx]

    #         if not isinstance(item, dict):
    #             raise TypeError(f"Expected dict but got {type(item)} at index {idx}")

    #         return {
    #             "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
    #             "attention_mask": torch.tensor(item["attention_mask"], dtype=torch.long),
    #             "labels": torch.tensor(item["labels"], dtype=torch.long) if "labels" in item else None
    #         }

    #     except Exception as e:
    #         print(f"Error at index {idx}: {e}")
    #         print(f"Batch content at index {idx}: {self.data[idx]}")
    #         return None


In [ ]:
# Example dataset
df = pd.read_csv('def_api.csv')
data = []
missed_data_points = []
for i,cla in enumerate(df['classes']):
  if isinstance(df['meainings'][i],str):
    data.append({'class':cla,'meaning':df['meainings'][i]})
  else:
    missed_data_points.append(cla)
print(data)

In [10]:
#tokenize dataset
dataset = WordMeaningDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [8]:
import torch.optim as optim
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define loss function
loss_fn = torch.nn.MSELoss()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Define dummy target labels (modify this for your task)
        target = torch.zeros_like(logits)

        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


/Users/rajshekhar/Desktop/ci/.venv/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 'meanings'

In [ ]:
model.save_pretrained("bert_word_meaning_model")
tokenizer.save_pretrained("bert_word_meaning_model")

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert_word_meaning_model")
tokenizer = BertTokenizer.from_pretrained("bert_word_meaning_model")

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class SentenceEncoder(nn.Module):
    def __init__(self, bert_model_name="bert_word_meaning_model", embed_dim=768, num_filters=128, kernel_size=3):
        super(SentenceEncoder, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)  # BERT as embedding layer
        self.conv1d = nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu = nn.ReLU()
        self.max_pool = nn.AdaptiveMaxPool1d(1)  # Global max pooling

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT during encoding
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        embeddings = bert_outputs.last_hidden_state  # (batch_size, seq_length, embed_dim)
        embeddings = embeddings.permute(0, 2, 1)  # Convert to (batch_size, embed_dim, seq_length) for CNN
        
        conv_output = self.conv1d(embeddings)  # Apply CNN
        conv_output = self.relu(conv_output)
        pooled_output = self.max_pool(conv_output).squeeze(-1)  # Get fixed-size sentence vector
        
        return pooled_output  # (batch_size, num_filters)

# Example usage
tokenizer = BertTokenizer.from_pretrained("bert_word_meaning_model")

def encode_sentence(sentence, max_length=128):
    tokens = tokenizer(sentence, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    return tokens["input_ids"], tokens["attention_mask"]

# Sample sentence with entity indicators
sentence = "[E1] Barack Obama [/E1] was the 44th president of the [E2] United States [/E2]."
input_ids, attention_mask = encode_sentence(sentence)

# Model forward pass
model = SentenceEncoder()
sentence_vector = model(input_ids, attention_mask)
print("Sentence representation shape:", sentence_vector.shape)  # Should be (1, num_filters)
